# Heisenberg Hamiltonian simulation

Quantum state preparation is a core component of useful quantum algorithms, but it is often one of the most complex and expensive elements to simulate. Keeping the same state in memory and reusing it can optimize simulations when the state is used multiple times with different parameters or following classical pre– and post-processing steps.

The state-handling tools can be especially helpful for quantum algorithms that are recursive or iterative in nature. 

Figure below represents an example case where subsequent quantum operations $U_{n+1}$ (green boxes) are determined classically based on measurement results from the previous state $(\psi_n)$. 

Without state handling, the n+1 step would require the simulation of all previous operations (gray boxes) to produce $\psi_n$  before $U_{n+1}$ is applied and the process continues. 

Retaining the previous state in GPU memory eliminates the need to perform all previous operations, significantly boosting performance

<div>
<img src="./state-handle.png" width="500">
</div>

Simulating Trotter dynamics is a great quantitative example of a state handling performance boost. In a 25-qubit benchmark of a Heisenberg Hamiltonian simulation, saving the previous state means each step takes approximately the same amount of time. Without state handling, the steps become increasingly onerous. For a simulation with 100 steps, state handling results in an impressive 24x faster total simulation time.

<div>
<img src="./perfomance.png" width="500">
</div>

In [4]:
import cudaq
import time
import numpy as np
from typing import List


In [5]:
spins = 11  
steps = 10  

cudaq.set_target("nvidia")


In [6]:
# Alternating up/down spins
@cudaq.kernel
def getInitState(numSpins: int):
    q = cudaq.qvector(numSpins)
    for qId in range(0, numSpins, 2):
        x(q[qId])


# This performs a single-step Trotter on top of an initial state, e.g.,
# result state of the previous Trotter step.
@cudaq.kernel
def trotter(state: cudaq.State, coefficients: List[complex],
            words: List[cudaq.pauli_word], dt: float):
    q = cudaq.qvector(state)
    for i in range(len(coefficients)):
        exp_pauli(coefficients[i].real * dt, q, words[i])


In [7]:
def run_steps(steps: int, spins: int):
    g = 1.0
    Jx = 1.0
    Jy = 1.0
    Jz = g
    dt = 0.05
    n_steps = steps
    n_spins = spins
    omega = 2 * np.pi

    def heisenbergModelHam(t: float) -> cudaq.SpinOperator:
        tdOp = cudaq.SpinOperator(num_qubits=n_spins)
        for i in range(0, n_spins - 1):
            tdOp += (Jx * cudaq.spin.x(i) * cudaq.spin.x(i + 1))
            tdOp += (Jy * cudaq.spin.y(i) * cudaq.spin.y(i + 1))
            tdOp += (Jz * cudaq.spin.z(i) * cudaq.spin.z(i + 1))
        for i in range(0, n_spins):
            tdOp += (np.cos(omega * t) * cudaq.spin.x(i))
        return tdOp

    # Collect coefficients from a spin operator so we can pass them to a kernel
    def termCoefficients(op: cudaq.SpinOperator) -> List[complex]:
        result = []
        ham.for_each_term(lambda term: result.append(term.get_coefficient()))
        return result

    # Collect Pauli words from a spin operator so we can pass them to a kernel
    def termWords(op: cudaq.SpinOperator) -> List[str]:
        result = []
        ham.for_each_term(lambda term: result.append(term.to_string(False)))
        return result

# Observe the average magnetization of all spins (<Z>)
    average_magnetization = cudaq.SpinOperator(num_qubits=n_spins)
    for i in range(0, n_spins):
        average_magnetization += ((1.0 / n_spins) * cudaq.spin.z(i))
    average_magnetization -= 1.0

    # Run loop
    state = cudaq.get_state(getInitState, n_spins)

    results = []
    times = []
    for i in range(0, n_steps):
        start_time = time.time()
        ham = heisenbergModelHam(i * dt)
        coefficients = termCoefficients(ham)
        words = termWords(ham)
        magnetization_exp_val = cudaq.observe(trotter, average_magnetization,
                                              state, coefficients, words, dt)
        result = magnetization_exp_val.expectation()
        results.append(result)
        state = cudaq.get_state(trotter, state, coefficients, words, dt)
        stepTime = time.time() - start_time
        times.append(stepTime)
        print(f"Step {i}: time [s]: {stepTime}, result: {result}")

    print("")
    # Print all the times and results (useful for plotting).
    print(f"Step times: {times}")
    print(f"Results: {results}")

    print("")

start_time = time.time()
run_steps(steps, spins)
print(f"Total time: {time.time() - start_time}s")


Step 0: time [s]: 0.048674583435058594, result: -0.09042024163828166
Step 1: time [s]: 0.002845287322998047, result: -0.08898564687193886
Step 2: time [s]: 0.0029337406158447266, result: -0.08698024360923415
Step 3: time [s]: 0.002657651901245117, result: -0.08507694741170907
Step 4: time [s]: 0.002768278121948242, result: -0.08394118068746997
Step 5: time [s]: 0.0027260780334472656, result: -0.08394076573115139
Step 6: time [s]: 0.002973794937133789, result: -0.08502222139504187
Step 7: time [s]: 0.0026557445526123047, result: -0.08677832064885871
Step 8: time [s]: 0.0027344226837158203, result: -0.08863390649349775
Step 9: time [s]: 0.0026917457580566406, result: -0.09005513983609514

Step times: [0.048674583435058594, 0.002845287322998047, 0.0029337406158447266, 0.002657651901245117, 0.002768278121948242, 0.0027260780334472656, 0.002973794937133789, 0.0026557445526123047, 0.0027344226837158203, 0.0026917457580566406]
Results: [-0.09042024163828166, -0.08898564687193886, -0.086980243